### Accessing the Jawbone API using OAuth2

Next, we will create the code that can interact with the Jawbone API. The Jawbone API documentation (https://jawbone.com/up/developer/) provides a [good explanation of the authentication process](https://jawbone.com/up/developer/authentication). 

Below, we implement the OAuth2 flow for the API. The code below will create the webserver that will handle the "callback" calls from the API. (Steps 3 and 4 from the figure above.) Then, we will show how to implement the rest of the flow.

In [1]:
# This part creates a temporary web server to "receive" the requests by Jawbone

# In this cell, we just configure the server. We will start it, 
# using the command webserver.run(host='0.0.0.0') in the next cell

# Flask is a webserver
from flask import Flask, request

# We will use that part to issue a request to Jawbone
import requests
import json



# Get these after registering an app with Jawbone
# These are to communicate to Jawbone that the requests come
# from a legitimate, registered app
CLIENT_ID = 'F3rPnz_fTUY'
CLIENT_SECRET = 'efaedc99a6bc5fb40f4be3b8fb97c6b94a7eec58'

# This is the location where we will store the authentication data from Jawbone
OAUTH_FILE = '/home/ubuntu/data/jawbone_secret.json'

# Initialize the embedded web server
webserver = Flask("JawboneOAuth")

# The function below will implement the sole functionality of our server.
# The Jawbone API will first authenticate the user and then direct the browser
# to http://<your ip>:5000/receiveCode in order to send the code to the app.
# 
# This is the place where the webserver will receive the call from Jawbone
# with the authentication code (Step 3 in https://jawbone.com/up/developer/authentication)
# Notice that we will use the http://<your ip>:5000/receiveCode address below as the redirect URI 
@webserver.route("/receiveCode")
def oauth_helper():
    code = request.args.get('code')
    
    # Now that we got the code, we move to Step 4 of the instructions
    # and request the access token from Jawbone. Notice that we 
    # use the client_secret to prove that the app is the real one
    # that was registered with the Jawbone API
    url = "https://jawbone.com/auth/oauth2/token"
    params = {"grant_type": "authorization_code", 
              "client_id": CLIENT_ID, 
              "client_secret": CLIENT_SECRET, 
              "code": code }
    resp = requests.get(url, params=params)
    data = json.loads(resp.text)
    
    # We store the code in a file as the webserver does not interact with the 
    # rest of the Python code, and we also want to reuse the code in the future
    # (Typically, we would store the access_token in a database.)
    f = open(OAUTH_FILE, 'w') # Store the code as a file
    f.write(resp.text)
    f.close()
    
    # It is safe (and convenient) to shut down the web server after this request
    stop_server()
    
    # What we return here has no real impact on the functionality of the code
    return '<html><body>Code: <b>'+code+'</b><p>Response:<b>'+resp.text+'</b></body></html>'


def start_server():
    webserver.run(host='0.0.0.0', port=5000)
    return
    
def stop_server():
    shutdown_after_request = request.environ.get('werkzeug.server.shutdown')
    shutdown_after_request()
    return
    


So our server is ready to run now, and it's ready to accept the requests from Jawbone. In principle, we can write the code in the cell above in a separate Python script, and let it run in the background.

Let's start the OAuth2 flow now, to first interact with the user and get the user to login and grant permissions. (Steps 1 and 2 of the figure above.)

In [4]:
# To get the server to run in the background
import threading

# This part is required to come up with the pop-up with which Jawbone will ask for permissions
from IPython.display import display
from IPython.display import Javascript as JS


# Send an OAuth request to Jawbone, handle the redirect, and display the access
# token that's included in the redirect for the user to copy and paste
REDIRECT_URI = 'http://ipython.ipeirotis.com:5000/receiveCode'
JAWBONE_URL = 'https://jawbone.com/auth/oauth2/auth'
PERMISSIONS = 'basic_read extended_read sleep_read move_read'

url = (JAWBONE_URL + 
       '?response_type=code&client_id='+ CLIENT_ID + 
       '&scope=' + PERMISSIONS +
       '&redirect_uri=' + REDIRECT_URI )

# The code below is necessary to pop up the login window from Jawbone
# The login still start in a separate thread, in order to allow the execution
# to continue, and run the "start_server()" call below.
threading.Timer(1, lambda: display(JS("window.open('%s')" % url))).start()

# And now start the webserver so that we can receive the answer from Jawbone API
start_server()

<IPython.core.display.Javascript object>

In [5]:
# Read the access token from the file
f = open(OAUTH_FILE, 'r') 
content = f.read()
f.close()
auth_info = json.loads(content)

auth_info

{u'access_token': u'aV1SI82xvTr8kBFGA3_2VAo-F5IbiT1zPxpUR_-PPzNC7LrLTyKlERwG5wFar8U_2d-rVHICSX6Fw6yPEhcRiFECdgRlo_GULMgGZS0EumxrKbZFiOmnmAPChBPDZ5JP',
 u'expires_in': 31536000,
 u'refresh_token': u'gkGL9TjxP67eAnSXM0K37py9XOpU5P3eut8Xi9a70xzdL2WxNgQvKekaCy5aBtavNNWfJhnfRQwlAN2iCODyqw',
 u'token_type': u'Bearer'}

At this point we are pretty much done. We only need to send standard API calls, as usual, with the only addition being that we include an extra _header_ in the request, with the access_token. The code below is just showing various use cases of the API.

In [13]:
# Testing another of the API endpoints
endpoint = "https://jawbone.com/nudge/api/v.1.1/users/@me/moves"

# See https://jawbone.com/up/developer/endpoints/moves for details
params = {}
headers = {'Authorization': 'Bearer ' + auth_info['access_token']}
resp = requests.get(endpoint, params=params, headers=headers)
moves_data = json.loads(resp.text)
moves_data

{u'data': {u'items': [{u'date': 20160128,
    u'details': {u'active_time': 3077,
     u'bg_calories': 254.392588615,
     u'bmr': 1616.73715633,
     u'bmr_day': 1689.66783661,
     u'calories': 316.789621046,
     u'distance': 3385.09702301,
     u'hourly_totals': {u'2016012800': {u'active_time': 63,
       u'calories': 8.0,
       u'distance': 112.0,
       u'inactive_time': 0,
       u'longest_active_time': 63,
       u'longest_idle_time': 0,
       u'steps': 160},
      u'2016012801': {u'active_time': 250,
       u'calories': 15.0,
       u'distance': 171.0,
       u'inactive_time': 0,
       u'longest_active_time': 187,
       u'longest_idle_time': 0,
       u'steps': 261},
      u'2016012808': {u'active_time': 0,
       u'calories': 8.0,
       u'distance': 67.0,
       u'inactive_time': 0,
       u'longest_active_time': 0,
       u'longest_idle_time': 0,
       u'steps': 100},
      u'2016012809': {u'active_time': 1195,
       u'calories': 91.0,
       u'distance': 1358.0,
     

In [14]:
import pandas as pd

df = pd.DataFrame(moves_data["data"]["items"])
df

,date,details,snapshot_image,time_completed,time_created,time_updated,title,type,xid
0,20160128,"{u'active_time': 3077, u'tzs': [[1453957259, u...",/nudge/image/e/1454022256/1zcGUfR4KpO3U2cUJkgt...,1454021541,1453957259,1454040003,"4,659 steps today",move,1zcGUfR4KpO3U2cUJkgtty7GBbIbPKmv
1,20160127,"{u'active_time': 4338, u'tzs': [[1453870813, u...",/nudge/image/e/1453957335/1zcGUfR4KpMvuN21ZQpf...,1453957196,1453870878,1454040003,"6,411 steps",move,1zcGUfR4KpMvuN21ZQpfBWf4KqtiTWX_
2,20160126,"{u'active_time': 5290, u'tzs': [[1453784457, u...",/nudge/image/e/1453872102/VPXBDG6bvPkN_8WJC1BX...,1453869732,1453786626,1454040003,"7,889 steps",move,VPXBDG6bvPkN_8WJC1BXF0W3AjZCcxW9
3,20160125,"{u'active_time': 4165, u'tzs': [[1453698030, u...",/nudge/image/e/1453868488/VPXBDG6bvPl94lylCQfh...,1453783129,1453698664,1454040003,"6,808 steps",move,VPXBDG6bvPl94lylCQfhOf8ZKFCHBGM1
4,20160124,"{u'active_time': 2517, u'tzs': [[1453611634, u...",/nudge/image/e/1453868487/VPXBDG6bvPmCMfdQf9yv...,1453697451,1453612590,1454040003,"4,977 steps",move,VPXBDG6bvPmCMfdQf9yvRNSK8Vaptd_5
5,20160123,"{u'active_time': 815, u'tzs': [[1453525261, u'...",/nudge/image/e/1453612598/VPXBDG6bvPmx0qQRsU0m...,1453609099,1453526469,1454040003,"2,311 steps",move,VPXBDG6bvPmx0qQRsU0mH3RXDZm-fPS-
6,20160122,"{u'active_time': 3331, u'tzs': [[1453438809, u...",/nudge/image/e/1453555703/VPXBDG6bvPl5hl8tDG4Z...,1453523858,1453447194,1454040003,"5,867 steps",move,VPXBDG6bvPl5hl8tDG4ZnBh4F_-XtoVw
7,20160121,"{u'active_time': 2891, u'tzs': [[1453352414, u...",/nudge/image/e/1453438902/VPXBDG6bvPkpB-tsFzXR...,1453436014,1453355321,1454040003,"5,052 steps",move,VPXBDG6bvPkpB-tsFzXRnKi75yFv9dbY
8,20160120,"{u'active_time': 2031, u'tzs': [[1453266034, u...",/nudge/image/e/1453385398/VPXBDG6bvPnGb8UFstK6...,1453352290,1453266099,1454040003,"3,530 steps",move,VPXBDG6bvPnGb8UFstK6VV3Q6_wBCs7i
9,20160119,"{u'active_time': 6812, u'tzs': [[1453179638, u...",/nudge/image/e/1453266865/VPXBDG6bvPkaH1dVylwZ...,1453265907,1453179765,1454040003,"11,944 steps",move,VPXBDG6bvPkaH1dVylwZbq5_cD77Pwy7


In [15]:
df_details = pd.DataFrame(list(df["details"]))
df_details

,active_time,bg_calories,bmr,bmr_day,calories,distance,hourly_totals,inactive_time,km,longest_active,...,steps_3am,sunrise,sunset,tz,tzs,wo_active_time,wo_calories,wo_count,wo_longest,wo_time
0,3077,254.392589,1616.737156,1689.667837,316.789621,3385.097023,"{u'2016012811': {u'distance': 199.726089478, u...",27873,3.385097,432,...,421,0,0,America/New_York,"[[1453957259, America/New_York]]",0,0,0,0,0
1,4338,347.113333,1664.114190,1664.114190,435.082519,4622.726089,"{u'2016012705': {u'distance': 35.0, u'active_t...",35798,4.622726,566,...,525,0,0,America/New_York,"[[1453870813, America/New_York]]",0,0,0,0,0
2,5290,406.997927,1644.826200,1644.826200,514.273609,5383.874720,"{u'2016012618': {u'distance': 339.0, u'active_...",34980,5.383875,814,...,109,0,0,America/New_York,"[[1453784457, America/New_York]]",0,0,0,0,0
3,4165,330.000000,1667.657642,1667.657642,414.462748,4616.000000,"{u'2016012510': {u'distance': 303.0, u'active_...",35332,4.616000,694,...,335,0,0,America/New_York,"[[1453698030, America/New_York]]",0,0,0,0,0
4,2517,290.825137,1701.095683,1701.095683,341.868350,3486.613384,"{u'2016012420': {u'distance': 0.0, u'active_ti...",37064,3.486613,439,...,74,0,0,America/New_York,"[[1453611634, America/New_York]]",0,0,0,0,0
5,815,146.000000,1735.629530,1735.629530,162.527874,1499.000000,"{u'2016012300': {u'distance': 76.0, u'active_t...",38850,1.499000,188,...,112,0,0,America/New_York,"[[1453525261, America/New_York]]",0,0,0,0,0
6,3331,330.000000,1684.623850,1684.623850,397.552060,4154.000000,"{u'2016012203': {u'distance': 7.0, u'active_ti...",37023,4.154000,569,...,48,0,0,America/New_York,"[[1453438809, America/New_York]]",0,0,0,0,0
7,2891,255.000000,1693.564856,1693.564856,313.629561,3510.000000,"{u'2016012101': {u'distance': 41.0, u'active_t...",38854,3.510000,1137,...,164,0,0,America/New_York,"[[1453352414, America/New_York]]",0,0,0,0,0
8,2031,182.635480,1711.023752,1711.023752,223.824651,2327.912889,"{u'2016012008': {u'distance': 84.7727899551, u...",38463,2.327913,569,...,323,0,0,America/New_York,"[[1453266034, America/New_York]]",0,0,0,0,0
9,6812,662.575385,1614.080962,1614.080962,800.725853,8858.517321,"{u'2016011919': {u'distance': 711.925041437, u...",34249,8.858517,631,...,270,0,0,America/New_York,"[[1453179638, America/New_York]]",0,0,0,0,0


In [16]:
# Testing another of the API endpoints
endpoint = "https://jawbone.com/nudge/api/v.1.1/users/@me/sleeps"

# See https://jawbone.com/up/developer/endpoints/sleeps
params = {}
headers = {'Authorization': 'Bearer ' + auth_info['access_token']}
resp = requests.get(endpoint, params=params, headers=headers)
sleep_data = json.loads(resp.text)
sleep_data

{u'data': {u'items': [{u'date': 20160128,
    u'details': {u'asleep_time': 1453962674,
     u'awake': 1171,
     u'awake_time': 1453987330,
     u'awakenings': 1,
     u'body': u'',
     u'duration': 25256,
     u'light': 12407,
     u'mind': 0,
     u'quality': u'',
     u'rem': 0,
     u'smart_alarm_fire': u'',
     u'sound': 11678,
     u'sunrise': None,
     u'sunset': None,
     u'tz': u'America/New_York'},
    u'shared': True,
    u'snapshot_image': u'/nudge/image/e/1453988115/1zcGUfR4KpPVnjNu7AXRxjCd4mG0SqYz/F3rPnz_fTUY.png',
    u'sub_type': 0,
    u'time_completed': 1453987330,
    u'time_created': 1453962074,
    u'time_updated': 1453988115,
    u'title': u'for 6h 41m',
    u'xid': u'1zcGUfR4KpPVnjNu7AXRxjCd4mG0SqYz'},
   {u'date': 20160127,
    u'details': {u'asleep_time': 1453874266,
     u'awake': 1144,
     u'awake_time': 1453901489,
     u'awakenings': 1,
     u'body': u'',
     u'duration': 27823,
     u'light': 13099,
     u'mind': 0,
     u'quality': u'',
     u'rem':

In [17]:
df = pd.DataFrame(sleep_data["data"]["items"])
df

,date,details,shared,snapshot_image,sub_type,time_completed,time_created,time_updated,title,xid
0,20160128,"{u'body': u'', u'sound': 11678, u'smart_alarm_...",True,/nudge/image/e/1453988115/1zcGUfR4KpPVnjNu7AXR...,0,1453987330,1453962074,1453988115,for 6h 41m,1zcGUfR4KpPVnjNu7AXRxjCd4mG0SqYz
1,20160127,"{u'body': u'', u'sound': 13580, u'tz': u'Ameri...",True,/nudge/image/e/1453907802/1zcGUfR4KpNZWXmYcU-S...,0,1453901489,1453873666,1453907802,for 7h 24m,1zcGUfR4KpNZWXmYcU-Sg_Ng_vXN5LUO
2,20160126,"{u'body': u'', u'sound': 15347, u'tz': u'Ameri...",True,/nudge/image/e/1453868491/1zcGUfR4KpNqk4PsU8L_...,0,1453816447,1453787698,1453868491,for 7h 40m,1zcGUfR4KpNqk4PsU8L_UVjQcOfdn3WX
3,20160125,"{u'body': 0, u'sound': 0, u'tz': u'America/New...",True,,0,1453729248,1453706348,1453961677,for 0m,VPXBDG6bvPn8UR0ff4_N4_xyZYte4R4c
4,20160124,"{u'body': 0, u'sound': 18511, u'smart_alarm_fi...",True,/nudge/image/e/1453644495/VPXBDG6bvPnbwcIkoUC6...,0,1453642881,1453614286,1453644495,for 7h 41m,VPXBDG6bvPnbwcIkoUC6XbUIJB0QYBge
5,20160123,"{u'body': u'', u'sound': 16080, u'smart_alarm_...",True,/nudge/image/e/1453564788/VPXBDG6bvPkT1eHUvZsQ...,0,1453554721,1453527723,1453564788,for 7h 19m,VPXBDG6bvPkT1eHUvZsQQ2oyOcf8qIdj
6,20160122,"{u'body': 0, u'sound': 16460, u'tz': u'America...",True,/nudge/image/e/1453467925/VPXBDG6bvPkzqrvjr2_w...,0,1453467592,1453436378,1453467925,for 8h 11m,VPXBDG6bvPkzqrvjr2_w1_rHy_mNoYfR
7,20160121,"{u'body': 0, u'sound': 10090, u'tz': u'America...",True,/nudge/image/e/1453385571/VPXBDG6bvPkuBb91Gjkm...,2,1453383010,1453357152,1453385571,for 6h 36m,VPXBDG6bvPkuBb91GjkmEzc3sJ5l6wbQ
8,20160121,"{u'body': 0, u'sound': 0, u'tz': u'America/New...",True,,0,1453355943,1453353311,1453468003,for 43m,VPXBDG6bvPkv2qsQKoBJv4FK8HLDrUuY
9,20160120,"{u'body': 0, u'sound': 16470, u'tz': u'America...",True,/nudge/image/e/1453347170/VPXBDG6bvPmJzEjETFz5...,2,1453297200,1453268085,1453347170,for 7h 48m,VPXBDG6bvPmJzEjETFz5m_N71kGQuDfJ


In [ ]:
df_details = pd.DataFrame(list(df["details"]))
df_details

In [ ]:
df_details["asleep_time"] = map(lambda x : datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'), df_details["asleep_time"])
df_details["awake_time"] = map(lambda x : datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'), df_details["awake_time"])
df_details